# Example usage

To use `instrumentum` in a project:

In [1]:
import instrumentum

print(instrumentum.__version__)

0.2.0


In [2]:
import instrumentum.time_series as ts
import pandas as pd

data_file = "./sample_data/time_series.csv"
data_df = pd.read_csv(data_file)

ts.ts_print(data_df, col_target="ACQUIRED", col_key="CUSTOMER_KEY")

Total unique keys that with a possitive value:  2
Total records of ckeys that with a possitive value:  13
Total unique keys with no possitive value:  2
Total records of customers with no possitive values:  10


In [146]:
import instrumentum.features.stepwise as sw
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

data_file = "./sample_data/simple.csv"
data_df = pd.read_csv(data_file)


def myfunc(X_train, y_train):

    model = DecisionTreeClassifier()
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    return cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv).mean()
      
sw.forward_stepwise(data_df.drop("target",axis=1), data_df['target'], n_combs=1, rounding=3, _scorer=myfunc)
#ts.ts_print(data_df, col_target="ACQUIRED", col_key="CUSTOMER_KEY")

Number of combinations:  1
Training shape:  (29, 13)
Label distribution: 
 1    15
0    14
Name: target, dtype: int64

Combinations to test: 13
Best score: 0.86, previous 0, columns added: ['a']
Best columns so far: ['a']

Combinations to test: 12
Best score: 0.933, previous 0.86, columns added: ['d']
Best columns so far: ['a', 'd']

Combinations to test: 11

Best score: 0.933, columns final: ['a', 'd']
